For exporting to PDF:
```bash
apt-get install pandoc 
```

Install dependencies:
```bash
pip install bokeh
pip install jupyter_bokeh
```


In [32]:

import pandas as pd
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import re

from numpy import NaN

results = "./full-stack-test/scorer_load_test_1000vus_45m_increased_timeout.csv"
df = pd.read_csv(results, low_memory=False)
if "tracked=true" in df.extra_tags.unique():
    df = df[df["extra_tags"]=="tracked=true"]
df["timestamp"] = pd.to_datetime(df.timestamp, unit='s')

output_notebook()

df

Loading BokehJS ...

,metric_name,timestamp,metric_value,check,error,error_code,expected_response,group,method,name,proto,scenario,service,status,subproto,tls_version,url,extra_tags,metadata
0,data_sent,2024-05-13 20:53:46,0.000000,NaN,NaN,NaN,NaN,::setup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,data_received,2024-05-13 20:53:46,0.000000,NaN,NaN,NaN,NaN,::setup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,iteration_duration,2024-05-13 20:53:46,0.001375,NaN,NaN,NaN,NaN,::setup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,vus,2024-05-13 20:53:46,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vus_max,2024-05-13 20:53:46,1000.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2967883,data_sent,2024-05-13 21:39:19,10662.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2967884,data_received,2024-05-13 21:39:19,3958.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2967885,data_sent,2024-05-13 21:39:19,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2967886,data_received,2024-05-13 21:39:19,0.000000,NaN,NaN,NaN,NaN,::teardown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:

def extract_url_prefix(url):
    if url is NaN or 'localhost' in url:
        return None
    pattern = r"(.*)(0x[a-fA-F0-9]{40})"
    match = re.match(pattern, url)
    return match.group(1) if match else url

df['normalized_url'] = df['url'].apply(extract_url_prefix)
df['normalized_url']

df = df[df['normalized_url'].notna()]

In [34]:
df.metric_name.unique()

array(['http_reqs', 'http_req_duration', 'http_req_blocked',
       'http_req_connecting', 'http_req_tls_handshaking',
       'http_req_sending', 'http_req_waiting', 'http_req_receiving',
       'http_req_failed'], dtype=object)

In [35]:
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [36]:
df[df.metric_name == "http_reqs"].groupby(by=["status", "normalized_url", "method"]).count()

metric_name  \
status normalized_url                                     method                
0.0    https://api.staging.scorer.gitcoin.co/account/n... GET            3392   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            9349   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            3730   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH          9335   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            2874   
       https://api.staging.scorer.gitcoin.co/passport-... GET            2523   
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET           17447   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET           79590   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET           20534   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH         73978   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET           21401   
       https://api.staging.scorer.gitcoin.co/passport-... GET           17871   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH             2   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET            3460   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            5664   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET              34   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH         11607   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET              24   
       https://api.staging.scorer.gitcoin.co/passport-... GET            3674   
504.0  https://api.staging.scorer.gitcoin.co/passport-... GET             229   

                                                                  timestamp  \
status normalized_url                                     method              
0.0    https://api.staging.scorer.gitcoin.co/account/n... GET          3392   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET          9349   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET          3730   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH        9335   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET          2874   
       https://api.staging.scorer.gitcoin.co/passport-... GET          2523   
200.0  https://api.staging.scorer.gitcoin.co/account/n... GET         17447   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         79590   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         20534   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH       73978   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET         21401   
       https://api.staging.scorer.gitcoin.co/passport-... GET         17871   
400.0  https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH           2   
502.0  https://api.staging.scorer.gitcoin.co/account/n... GET          3460   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET          5664   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            34   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH       11607   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET            24   
       https://api.staging.scorer.gitcoin.co/passport-... GET          3674   
504.0  https://api.staging.scorer.gitcoin.co/passport-... GET           229   

                                                                  metric_value  \
status normalized_url                                     method                 
0.0    https://api.staging.scorer.gitcoin.co/account/n... GET             3392   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET             9349   
       https://api.staging.scorer.gitcoin.co/ceramic-c... GET             3730   
       https://api.staging.scorer.gitcoin.co/ceramic-c... PATCH           9335   
       https://api.staging.scorer.gitcoin.co/ceramic

In [37]:
# df['normalized_url'] = df.url.str.extract(r'(?<=http://localhost:8080)(.*)')
df.columns

Index(['metric_name', 'timestamp', 'metric_value', 'check', 'error',
       'error_code', 'expected_response', 'group', 'method', 'name', 'proto',
       'scenario', 'service', 'status', 'subproto', 'tls_version', 'url',
       'extra_tags', 'metadata', 'normalized_url'],
      dtype='object')

In [38]:
df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    # "slate gray",
    # "forest green",
    "lavender",
    "coral",
    "turquoise",
    "gold"
]


color = iter(colors)
p = figure(title="Duration (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        # p.line(group.index, group["metric_value"]["min"], legend_label=f"min - {url}", line_width=2, color=next(color))
        # p.line(group.index, group["metric_value"]["max"], legend_label=f"max - {url}", line_width=2, color=next(color))
        p.line(group.index, group["metric_value"]["mean"], legend_label=f"mean - {url}", line_width=2, color=next(color))


show(p)


In [39]:

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]


p = figure(title="Req / second (status == 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    if int(status) == 200:
        print(idx)
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])


show(p)

1
4
7
10
14
17


In [40]:

from math import pi
from bokeh.models import DatetimeTickFormatter
from collections import defaultdict

df_duration = df[df.metric_name == "http_req_duration"]

dft = df_duration.set_index("timestamp")

# dfts = dft.groupby(by=["url"]).resample("1Min").agg({
#     "metric_value": ["min", "max", "mean", "count"]
# })

dfts = dft.groupby(by=["normalized_url", "status"]).resample("s").agg({
    "metric_value": ["min", "max", "mean", "count"]
})

dfts.reset_index(inplace=True)
dfts.set_index("timestamp", inplace=True)

colors = [
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
    "blue",
    "red",
    "green",
    "purple",
    "orange",
    "teal",
    "pink",
    "yellow",
    "cyan",
    "maroon",
    "olive",
    "navy",
    "magenta",
    "brown",
    "brown",
    "red",
    "lavender",
    "coral",
    "turquoise",
    "gold",
]

p = figure(title="Req / second (status != 200)", x_axis_label='x', y_axis_label='y', frame_width=1500,
           tools="pan,wheel_zoom,box_zoom,reset,hover,crosshair",
           x_axis_type="datetime")

p.xaxis.major_label_orientation = pi/4

# Initialize dictionaries for tracking sums per URL
url_200_sums = defaultdict(int)
url_non_200_sums = defaultdict(int)

# Loop through the grouped data and collect sums for each URL
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    # Sum up counts for each status
    total_count = group["metric_value"]["count"].sum()

    # Were unable to get 200 for auth step so expecting it to return a 400
    if url == 'https://api.staging.scorer.gitcoin.co/ceramic-cache/authenticate' and status == 400:
        url_200_sums[url] += total_count

    
    # Check if the status is 200 or not and add to the appropriate sum
    if int(status) == 200:
        url_200_sums[url] += total_count
    else:
        url_non_200_sums[url] += total_count

# Print summary for each individual URL
for url in url_200_sums.keys() | url_non_200_sums.keys():
    sum_200 = url_200_sums[url]
    sum_non_200 = url_non_200_sums[url]
    total = sum_200 + sum_non_200
    success_rate = (float(sum_200) / total if total > 0 else 0) * 100

    print(f"URL: {url}")
    print(f"   200       {sum_200}")
    print(f"   non-200   {sum_non_200}")
    print(f"   success {success_rate:.2f}%\n")


    # Number of Passports Created
    # For each VU 4 requests are made to /ceramic-cache/stamps/bulk and /ceramic-cache/score/

    if "stamps/bulk" in url:
        print(f"   Passports Created: {sum_200 / 4}\n")

    if "ceramic-cache/score/" in url:
        print(f"   Passports Scored: {sum_200 / 4}\n")



# Optional: If you also want to show the plots
for idx, ((url, status), group) in enumerate(dfts.groupby(by=["normalized_url", "status"])):
    print(idx)
    if int(status) != 200:
        p.line(group.index, group["metric_value"]["count"], legend_label=f"{int(status)} - {url}", line_width=2, color=colors[idx])

show(p)

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/stamps/bulk
   200       73978
   non-200   20944
   success 77.94%

   Passports Created: 18494.5

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/stamp?address=
   200       20534
   non-200   3764
   success 84.51%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/score/
   200       79590
   non-200   15013
   success 84.13%

   Passports Scored: 19897.5

URL: https://api.staging.scorer.gitcoin.co/account/nonce
   200       17447
   non-200   6852
   success 71.80%

URL: https://api.staging.scorer.gitcoin.co/passport-admin/banners
   200       17871
   non-200   6426
   success 73.55%

URL: https://api.staging.scorer.gitcoin.co/ceramic-cache/weights
   200       21401
   non-200   2898
   success 88.07%

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
